## Settings

### Importing libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime as dt

from scipy import stats
from scipy.stats import mode

# !pip install pandasql
from pandasql import sqldf

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio

# to print all the outputs in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# to show warnings only once:
import warnings; warnings.filterwarnings(action='once')

# import the own module 
import helpers

### Some additional settings

In [ ]:
# adjusting columns width & number of chars
pd.set_option('display.width', 600)
pd.set_option('max_colwidth', 500)

# set pandas max columns and rows to print
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# settings for Vizs
large = 22
med = 16
small = 12
params = {
    'axes.titlesize': large,
    'legend.fontsize': med,
    'figure.figsize': (17, 10),
    'axes.labelsize': med,
    'axes.titlesize': med,
    'xtick.labelsize': med,
    'ytick.labelsize': med,
    'figure.titlesize': large
}
plt.rcParams.update(params)
sns.set_style("white")
%matplotlib inline

*Selecting theme for graphs*

In [ ]:
pio.templates

In [ ]:
pio.templates.default = "plotly_dark"

px.defaults.template = "plotly_dark"
px.defaults.color_continuous_scale = px.colors.sequential.gray_r
# px.defaults.width = 600
# px.defaults.height = 400
# color_discrete_sequence=px.colors.sequential.Oryel

## Download the data and prepare it for analysis.

In [ ]:
source_dir = './data/raw/' 

missing_values = ["n/a", "na", "--"]

# files to download:
raw_visits = 'visits_log_us.csv'
raw_orders = 'orders_log_us.csv'
raw_costs = 'costs_us.csv'

### The `visits_log_us` dataset

_Downloading a small portion of data for the first look at the data_

In [ ]:
visits_portion = pd.read_csv(os.path.join(source_dir, raw_visits),
                          parse_dates=['Start Ts', 'End Ts'],
                          na_values=missing_values,
                          nrows=500)

visits_portion.info(memory_usage='deep')
visits_portion.head()

*Downloading data with the correct data types*

In [ ]:
df_visits_origin = pd.read_csv(os.path.join(source_dir, raw_visits),
                     na_values=missing_values,
                     dtype={'Device': 'category', 'Source Id': 'category', 'Uid': 'string'},
                     parse_dates=['Start Ts', 'End Ts'])

In [ ]:
# saving the original data set for comparing changes to be made
df_visits = df_visits_origin.copy()

#### Summary statistics

In [ ]:
df_visits.shape

In [ ]:
df_visits.info(memory_usage='deep')

In [ ]:
# df_visits['Uid'] = df_visits['Uid'].astype('string')
# df_visits.info()

In [ ]:
df_visits.describe()

In [ ]:
df_visits.head()
df_visits.tail()

In [ ]:
df_visits.isna().mean()

In [ ]:
df_visits.duplicated().mean()

In [ ]:
df_visits.nunique()

#### Data pre-processing

*Converting column names to lowercase and replacing space with `_`:*

In [ ]:
df_visits.columns = df_visits.columns.str.lower().str.replace(' ', '_')
df_visits.head()

*Rename some columns for better understanding data by business users*

In [ ]:
df_visits.rename(columns={'end_ts': 'session_end_ts',
                          'start_ts': 'session_start_ts'}, inplace=True)

 *Replacing the value `touch` with `mobile` and the value `desktop` with `browser` according to the project description.*

In [ ]:
df_visits = df_visits.replace({'touch': 'mobile', 'desktop': 'browser'})
df_visits['device'].unique()

*Reoderring columns*

In [ ]:
df_visits = df_visits.reindex(columns=['uid', 'device', 'source_id', 'session_start_ts', 'session_end_ts'])

In [ ]:
df_visits.sample(5)

*Are there entries outside of our period from June 1, 2017 to May 31, 2018*

In [ ]:
df_visits[df_visits['session_end_ts'] >= '2018-06-01'].shape[0]
df_visits[df_visits['session_end_ts'] >= '2018-06-01']

>The DataFrame contains 12 records with the `session_end_ts` out of scope. however, these sessions started within researching period. 

So, before deleting these records, let's find out if users made any purchases during these sessions. We will verify this in the next step by analyzing the orders table.

*Are there weird entries where `start_ts` is bigger than `end_ts`:*

In [ ]:
df_visits[df_visits['session_start_ts'] > df_visits['session_end_ts']]

> We will handle these 2 cases after detecting user patterns using the order dataset:
> 
- Did they make any purchases? 
- Maybe some sites use Afisha like a gate for payments , etc

>**Conclusion:**

- The `visits` table has 359,400  entries. 
- There is no missing values 
- There is no duplicated values 
- Users come from 9 `source_id`
- Users used 2 types of devices, browser and mobile
- There are 12 entries with `session_end_ts` out of scope
- There are 2 entries with `session_start_ts` latter than `session_end_ts

(*NB! in this version I did not remove outliers on this step!*)

### The `orders_log_us` dataset

*Downloading the portion of data*

In [ ]:
orders_portion = pd.read_csv(os.path.join(source_dir, raw_orders),
                          # parse_dates=['End Ts', 'Start Ts'],
                          na_values=missing_values,
                          nrows=100)

orders_portion.info(memory_usage='deep')
orders_portion.head()

*Downloading the the full orders dataset with correct datatype*

In [ ]:
orders_orig = pd.read_csv(os.path.join(source_dir, raw_orders),
                          dtype={'Uid': 'string'},
                          parse_dates=['Buy Ts'],
                          )
# saving the original data set for comparing changes to be made
df_orders = orders_orig.copy()

#### Summary statistics

In [ ]:
df_orders.shape
df_orders.info()
df_orders.head()
df_orders.tail()
df_orders.sample(5)

In [ ]:
df_orders.describe()

In [ ]:
df_orders.isna().mean()

In [ ]:
df_orders.duplicated().mean()

#### Data pre-processing

*Converting column names to lowercase and replacing space with `_`:*

In [ ]:
df_orders.columns = df_orders.columns.str.lower().str.replace(' ', '_')
df_orders.head()

*Reordering columns*

In [ ]:
df_orders = df_orders.reindex(columns=['uid', 'buy_ts', 'revenue'])
df_orders.head()

*Is `buy_ts` in scope of the project?*

In [ ]:
df_orders[df_orders['buy_ts'] >= '2018-06-01 00:00:00']

> * In the `order` table, we only have 1 record, which is not included in our annual period. It should be deleted.

In [ ]:
df_orders = df_orders[df_orders['buy_ts'] < '2018-06-01 00:00:00']

# verifying result
df_orders[df_orders['buy_ts'] >= '2018-06-01 00:00:00']
df_orders.buy_ts.max()

*Does dataset contain Zero revenue?*

In [ ]:
len(df_orders[df_orders['revenue'] <= 0])

*The `revenue` variable contains 51 rows with zero value. It seems that these rows have no value for analysis and should be removed from the table.*

Well, they could have value, because free tickets still count in conversion. But there are very few of them, so we can remain them in this case.

#### Distribution of revenue

In [ ]:
df_orders['revenue'].mode()

In [ ]:
df_orders['revenue'].skew(skipna=True)

In general, revenue per purchasing is 1.83. *The revenue variable has a huge positive skewness 58.886451.*

*Revenue distribution plot*

In [ ]:
fig = px.histogram(df_orders, 
                   x="revenue", 
                   nbins=20,
                   title='Distribution of revenue',
                   color_discrete_sequence=px.colors.sequential.Oryel
                  )
fig.show()

#### Examine outliers

In [ ]:
fig = px.box(df_orders, 
                   x="revenue",
                   title='Distribution of revenue',
                   color_discrete_sequence=px.colors.sequential.Oryel
                  )
fig.show()

*Replacing `income` values of 0.95 quantiles with the most frequently occurring value*

In [ ]:
quantile_val = df_orders['revenue'].quantile(0.95)
quantile_val

mode_val = df_orders['revenue'].mode()
mode_val 

In [ ]:
df_orders['revenue'] = np.where(
    df_orders['revenue'] > quantile_val, mode_val, df_orders['revenue'])

In [ ]:
fig = px.box(df_orders, 
                   x="revenue",
                   title='Distribution of revenue',
                   color_discrete_sequence=px.colors.sequential.Oryel
                  )
fig.show()

In [ ]:
df_orders['revenue'].describe()

In [ ]:
df_orders[df_orders['revenue'] > 9].shape[0] / df_orders.shape[0]

On the one hand, an average `revenue` from purchase is 3.22, and mode is 1.83. And as many as 13.75 (max `revenue`) may seem like weird value. And as the boxplot shows, we can decide that all revenues above 9 are emissions (3.25 % of dataset).

On the other hand, in order to decide whether they are outliers or not and remove them, I would like to know more about the business domain. Now let's remain them in the dataset.

>Both datasets, `visits` and `orders`, lack the very important "id truncation" fields. 

However, we have to ensure that all users who have purchased products are represented in the `visits` table.

In [ ]:
df_orders[~df_orders['uid'].isin(df_visits['uid'])]

*All the buyers are represented in the `visit` table*

### Merging the `visits` and `orders` dataframes using pandasql library

https://pypi.org/project/pandasql/

In [ ]:
q = """SELECT V.uid,
V.device,
V.source_id, 
V.session_start_ts, 
V.session_end_ts,
O.buy_ts, 
O.revenue
FROM df_visits as V 
LEFT JOIN df_orders as O 
ON V.uid = O.uid
AND O.buy_ts BETWEEN V.session_start_ts AND V.session_end_ts
"""

In [ ]:
_df_visits_left = sqldf(q)
_df_visits_left.sample(10)

In [ ]:
q = """SELECT O.uid, 
V.device,
V.source_id, 
V.session_start_ts, 
V.session_end_ts,
O.buy_ts, 
O.revenue
FROM df_orders as O  
LEFT JOIN df_visits as V 
ON V.uid = O.uid
AND O.buy_ts BETWEEN V.session_start_ts AND V.session_end_ts
"""

In [ ]:
_df_orders_left = sqldf(q)
_df_orders_left.sample(10)

In [ ]:
df_users = pd.concat([_df_visits_left,_df_orders_left], ignore_index=True).drop_duplicates()

In [ ]:
df_users.sample(10)

*Verifying the control sums after merging DataFrames*

In [ ]:
df_orders['revenue'].sum()
df_users['revenue'].sum()

In [ ]:
df_visits['uid'].count()
df_users['uid'].count()

df_visits['uid'].nunique()
df_users['uid'].nunique()

In [ ]:
df_visits['session_start_ts'].count()

df_users['session_start_ts'].count()

In [ ]:
df_users.duplicated().sum()

#### *Handling null values in the DataFrame `df_users`*

In [ ]:
df_users.isna().mean()

In [ ]:
df_users['revenue'] = df_users['revenue'].fillna(0)

*As mentioned above, there are users who made purchases through sites using Afisha as a payment gateway.*

*In this case we can fill Null values in `session_start_ts` and in `session_end_ts` with the `buy_ts`*

In [ ]:
df_users['session_start_ts'] = df_users['session_start_ts'].fillna(df_users['buy_ts'])
df_users['session_end_ts'] = df_users['session_end_ts'].fillna(df_users['buy_ts'])

*Filling Null of `device` with values `unknown`*

In [ ]:
df_users['device'].fillna('unknown', inplace=True)

*Filling Null of `source_id` with any other unique value*

In [ ]:
df_users['source_id'].unique()

In [ ]:
df_users['source_id'].fillna(0, inplace=True)

In [ ]:
df_users['source_id'].unique()

In [ ]:
df_users.isna().mean()

*`buy_ts` will be left with Null values for obvious reasons*

#### *Handling data types*

In [ ]:
df_users.info()

*Converting `device` and `source_id` values to category*

In [ ]:
df_users['device'] = helpers.dtype_converter(df_users['device'], 'category')
df_users['source_id'] = helpers.dtype_converter(df_users['source_id'], 'category')

*Converting datetime object to datetime format*

In [ ]:
df_users['session_start_ts'] =pd.to_datetime(df_users['session_start_ts'], format='ISO8601')
df_users['session_end_ts'] =pd.to_datetime(df_users['session_end_ts'], format='ISO8601')
df_users['buy_ts'] =pd.to_datetime(df_users['buy_ts'], format='ISO8601')

In [ ]:
df_users.info()

In [ ]:
df_users.sample(10)

*Processing records outside the scope of the project*

In [ ]:
df_users[(df_users['session_start_ts'] > '2018-06-01') | (df_users['session_end_ts'] > '2018-06-01') | (df_users['buy_ts'] > '2018-06-01')]

> There are 11 entries in the `df_users` table where `sesstion_end_ts` is out of scope for the project.

Note! During 3 of them, purchases were made within our project time. 

It is crucial to not ignore them. All other inputs can be eliminated.


In [ ]:
# records to be deleted

df_users[(df_users['session_end_ts'] > '2018-06-01') & (df_users['buy_ts'].isna())]

In [ ]:
indx = df_users.index[(df_users['session_end_ts'] > '2018-06-01') & (df_users['buy_ts'].isna())].tolist()
indx

In [ ]:
df_users = df_users[~df_users.index.isin(indx)]

In [ ]:
# verifying the result

df_users[(df_users['session_end_ts'] > '2018-06-01') & (df_users['buy_ts'].isna())].shape[0]

### Adding the new column `session_duration_sec` to the table

In [ ]:
df_users['session_duration_sec'] = helpers.time_diff(df_users['session_end_ts'], df_users['session_start_ts'], 'seconds')

In [ ]:
df_users.sample(10)
df_users['session_duration_sec'].mode()
df_users['session_duration_sec'].describe()
df_users['session_duration_sec'].skew(skipna=True)
(df_users.query('session_duration_sec == 0')).shape[0] / df_visits.shape[0]

*The most common session duration value is 60 seconds, the median value is 618.55 and the positive skewness is 8.53.*

*13.99% of sessions are 0 seconds long.*

*Maximum of duration - 84480 seconds that is equal to 23 hours and more...* 🤔

*And, we have a big positive skewness. That means there are the outliers. It is important to understand if they need to be removed or corrected.* 

*It would be useful to study the distribution of this quantity.*

In [ ]:
fig = px.histogram(df_users, 
                   x="session_duration_sec", 
                   nbins=30,
                   title='Distribution of session duration in seconds',
                   color_discrete_sequence=px.colors.sequential.Oryel
                  )
fig.show()

In [ ]:
fig = px.box(df_users, 
                   x="session_duration_sec",
                   y='device',
                   title='Distribution of session duration in seconds',
                   color_discrete_sequence=px.colors.sequential.Oryel
                  )
fig.show()

#### Replacing outlieers with the median

https://www.pluralsight.com/guides/cleaning-up-data-from-outliers

In [ ]:
quantile_95_session = df_users['session_duration_sec'].quantile(0.95).astype('int')
quantile_95_session

In [ ]:
median_session_duration = df_users['session_duration_sec'].median().astype('int')
median_session_duration

In [ ]:
df_users['session_duration_sec'] = np.where(
    df_users['session_duration_sec'] > quantile_95_session,
    median_session_duration,
    df_users['session_duration_sec'])

In [ ]:
fig = px.box(df_users, 
                   x="session_duration_sec",
                   y='device',
                   title='Distribution of session duration in seconds',
                   color_discrete_sequence=px.colors.sequential.Oryel
                  )
fig.show()

*Summary statistics of `df_users` dataframe*

In [ ]:
df_users.describe()
df_users.info()

>Conclusion:

- After preprocessing the `df_users` datasets has 373880 entries.
- There are 50414 purchases. 
- There are no duplicates.
- The Null values are presented only in the `buy_ts` series.
- There are no values out of scope of the priject.

### The `costs_us` dataset

In [ ]:
df_costs_origin = pd.read_csv(os.path.join(source_dir, raw_costs),
                    na_values=missing_values,
                    parse_dates=['dt'])

# saving the original data set for comparing changes to be made
df_costs = df_costs_origin.copy()

#### Summary statistics

In [ ]:
df_costs.info()
df_costs.head()
df_costs.tail()
df_costs.sample(5)

In [ ]:
df_costs.describe()

In [ ]:
df_costs.isna().mean()

In [ ]:
df_costs.duplicated().mean()

In [ ]:
df_costs.source_id.nunique()
df_costs.source_id.value_counts()

>The `costs` data contains Marketing expenses statistics
<br> The table has 2542 records of information on the expenses on the ad source on a specific day.
<br>There is no Nal and duplicated values.

#### Data pre-processing

*To make the dataset variables more readable let's rename `dt` column:*

In [ ]:
df_costs.head()

In [ ]:
df_costs.rename(columns={'dt': 'costs_date'}, inplace=True)
df_costs.head()

In [ ]:
fig = px.histogram(df_costs, 
                   x="costs", 
                   nbins=20,
                   title='Distribution of costs',
                   color_discrete_sequence=px.colors.sequential.Oryel
                  )
fig.show()

In [ ]:
_costs = df_costs.groupby('source_id')['costs'].sum().reset_index()
_costs

In [ ]:
fig = px.bar(_costs.sort_values(by='source_id', ascending=True),                    
                   x='source_id',
                   y="costs",
                   title='Distribution of costs by source_id',
                   color_discrete_sequence=px.colors.sequential.Oryel
                  )
fig.show()

*There are costs associated with 7 ID sources. The greatest expense was incurred by ID source #3, while the smallest expenses were for ID sources #9 and #10.*

In [ ]:
df_users.sample(5)

>Conclusion:

- The `costs` dataset has 2482 entries.
- More than 3/4 of all costs are allocated between the 3rd, 4th, and 5th "source_id". Moreover, the 3rd `source_id` costs more than the 4th and 5th together.
- There is no expense on the 6th, 7th, and 8th `source_id`.
- There is no duplicates in the table.

### Save pre-processed data to csv

In [ ]:
path_to_save = './data/processed/'
file_users = 'users_processed.csv'
file_costs = 'costs_processed.csv'

In [ ]:
df_users.to_csv(os.path.join(path_to_save, file_users)
                ,sep=','
                ,encoding='utf-8'
                ,index=False)

In [ ]:
df_costs.to_csv(os.path.join(path_to_save, file_costs), sep=',', encoding='utf-8', index=False)

## End